In [26]:
import pymongo
import pandas as pd
import re
import nltk
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [27]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
sentenceDB  = client["Sentencias"]
sentenceCollection = sentenceDB["Sentencias Judiciales"]
wordCollection = sentenceDB["WordTokens"]
oracionCollection = sentenceDB["oracionesTokens"]

In [28]:
data = sentenceCollection.find()
df = pd.DataFrame(list(data))

In [29]:
paradaEspanol=nltk.corpus.stopwords.words('spanish')
extraStopWords = ["''","``",'en','el','que','la']
StopWords = paradaEspanol+ list(string.punctuation)+extraStopWords


In [30]:
docs=df['sentencia'].tolist()

In [31]:
cv=CountVectorizer(max_df=0.85,stop_words=StopWords,max_features=10000)
word_count_vector=cv.fit_transform(docs)
feature_names=cv.get_feature_names()

In [32]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [33]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

In [34]:
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []

    for idx, score in sorted_items:
        fname = feature_names[idx]
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [35]:
def get_keywords(idx):

    #generate tf-idf for the given document
    tf_idf_vector=tfidf_transformer.transform(cv.transform([docs[idx]]))

    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only the top n; n here is 10
    keywords=extract_topn_from_vector(feature_names,sorted_items,10)
    
    return keywords

def print_results(idx,keywords):
    for k in keywords:
        print(k,keywords[k])

In [36]:
keyWordCollection = sentenceDB["Keywords"]

In [37]:
for idx,doc in enumerate(docs):
    mydict = { "_id": df['_id'][idx], "keyWords": get_keywords(idx) }
    x = keyWordCollection.insert_one(mydict)